In [1]:
import pandas as pd

gj_range = pd.read_csv('Gyeongju.csv')

In [2]:
keywords = ['여행', '분위기', '맛집', '감성', '체험', '가볼만한 곳']
categorys = ['CT1', 'AT4', 'CE7']
categorys_info = {'CT1' : '문화시설', 'AT4' : '관광명소', 'CE7' : '카페'}

In [3]:
gj = gj_range['이름']

In [4]:
import json
import requests

def search_result(keyword, category, gyeongju_name):
    result = []
    # REST 키
    headers = {"Authorization": "KakaoAK 7d323e26f84156c6194a9d77f33e1e6a"}
    # 파라미터
    params = {"query" : f"경주시 {gyeongju_name}{keyword}", "page" : 1, "category_group_code" : f"{category}"}
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"

    while True:
        # GET을 이용하여 획득
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            # Json을 이용하여 해제
            doc = json.loads(res.text)
            result.extend(doc['documents'])
            if doc['meta']['is_end'] == True:
                break
            else:
                params['page'] += 1
    return result

# GET을 이용하여 획득
#res = requests.get(url, headers=headers, params=params)
# Json을 이용하여 해제
#doc = json.loads(res.text)
# 200일 경우 정상
#res.status_code

In [5]:
import pandas as pd
from tqdm.notebook import tqdm

def search_df():
    results = []
    for idx in tqdm(gj):
        for category in categorys:
            for key in keywords:
                r = pd.DataFrame(search_result(key, category, idx))
                r['keyword'] = key
                results.append(r.copy())
    return pd.concat(results).reset_index(drop=True)

In [6]:
gyeongju_poi = search_df()
gyeongju_poi.to_csv('gyeongju_poi.csv',index=False)

  0%|          | 0/22 [00:00<?, ?it/s]

In [7]:
gyeongju_poi.head(2)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,체험,경북 경주시 감포읍 오류리 583-3,CT1,문화시설,"문화,예술 > 문화시설 > 전시관",,1394788185,,송대말등대 빛체험전시관,http://place.map.kakao.com/1394788185,경북 경주시 감포읍 감포로 226-19,129.5113590714177,35.807336332617545
1,여행,경북 경주시 감포읍 오류리,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",,11256465,,오류고아라해변,http://place.map.kakao.com/11256465,,129.509520717781,35.8213166189963


In [8]:
import pandas as pd

api_poi = pd.read_csv('gyeongju_poi.csv', index_col=False)
api_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969 entries, 0 to 2968
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              2969 non-null   object 
 1   address_name         2969 non-null   object 
 2   category_group_code  2969 non-null   object 
 3   category_group_name  2969 non-null   object 
 4   category_name        2969 non-null   object 
 5   distance             0 non-null      float64
 6   id                   2969 non-null   int64  
 7   phone                702 non-null    object 
 8   place_name           2969 non-null   object 
 9   place_url            2969 non-null   object 
 10  road_address_name    1208 non-null   object 
 11  x                    2969 non-null   float64
 12  y                    2969 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 301.7+ KB


In [9]:
pd.DataFrame(api_poi['keyword'].value_counts())

,keyword
여행,994
가볼만한 곳,990
맛집,966
감성,17
체험,2


In [10]:
pd.DataFrame(api_poi['category_group_name'].value_counts())

,category_group_name
관광명소,1980
카페,987
문화시설,2


In [11]:
included = {}
del_index = []
for idx, row in api_poi.iterrows():
    id = row['id']
    temp = api_poi[api_poi['id'] == id].copy()
    cnt = len(temp)
    if id not in included and cnt > 1:
        included[id] = True
        for i, r in temp.iterrows():
            if idx == i:
                continue
            else:
                api_poi.loc[idx, 'keyword'] = api_poi.loc[idx, 'keyword'] + ',' + r['keyword']
                del_index.append(i)

In [12]:
# 중복 id의 수, 삭제할 인덱스의 수
len(included.keys()), len(del_index)

(858, 1346)

In [13]:
# 중복 id의 인덱스 삭제
api_poi_del = api_poi.drop(del_index, axis=0)

In [14]:
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 0 to 2967
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1623 non-null   object 
 1   address_name         1623 non-null   object 
 2   category_group_code  1623 non-null   object 
 3   category_group_name  1623 non-null   object 
 4   category_name        1623 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1623 non-null   int64  
 7   phone                553 non-null    object 
 8   place_name           1623 non-null   object 
 9   place_url            1623 non-null   object 
 10  road_address_name    932 non-null    object 
 11  x                    1623 non-null   float64
 12  y                    1623 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 177.5+ KB


In [15]:
api_poi_del.to_csv('gyeongju_poi2.csv', index=False)

In [16]:
# id 중복 제거 데이터 확인(제주도_POI(API)4)
import pandas as pd

api_poi_del = pd.read_csv('gyeongju_poi2.csv', index_col=False)
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1623 non-null   object 
 1   address_name         1623 non-null   object 
 2   category_group_code  1623 non-null   object 
 3   category_group_name  1623 non-null   object 
 4   category_name        1623 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1623 non-null   int64  
 7   phone                553 non-null    object 
 8   place_name           1623 non-null   object 
 9   place_url            1623 non-null   object 
 10  road_address_name    932 non-null    object 
 11  x                    1623 non-null   float64
 12  y                    1623 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 165.0+ KB


In [17]:
len(api_poi_del['id'].unique())

1623

In [18]:
pd.DataFrame(api_poi_del['category_group_name'].value_counts())

,category_group_name
카페,868
관광명소,753
문화시설,2


In [19]:
api_poi_del

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,체험,경북 경주시 감포읍 오류리 583-3,CT1,문화시설,"문화,예술 > 문화시설 > 전시관",NaN,1394788185,NaN,송대말등대 빛체험전시관,http://place.map.kakao.com/1394788185,경북 경주시 감포읍 감포로 226-19,129.511359,35.807336
1,"여행,가볼만한 곳",경북 경주시 감포읍 오류리,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,11256465,NaN,오류고아라해변,http://place.map.kakao.com/11256465,NaN,129.509521,35.821317
2,"여행,가볼만한 곳",경북 경주시 감포읍 나정리 1,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,8223627,NaN,나정고운모래해변,http://place.map.kakao.com/8223627,경북 경주시 감포읍 동해안로 1915,129.491499,35.782290
3,"여행,가볼만한 곳",경북 경주시 감포읍 전촌리 1236,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,10714629,NaN,전촌솔밭해변,http://place.map.kakao.com/10714629,NaN,129.491030,35.786112
4,"여행,가볼만한 곳",경북 경주시 감포읍 감포리 1,AT4,관광명소,"여행 > 관광,명소 > 테마거리",NaN,2087577487,NaN,감포해국길,http://place.map.kakao.com/2087577487,경북 경주시 감포읍 감포로 119-6,129.501883,35.804818
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,맛집,경북 경주시 신평동 355,CE7,카페,음식점 > 카페,NaN,369713880,NaN,카페노프랭크,http://place.map.kakao.com/369713880,경북 경주시 천북남로 16,129.289116,35.844502
1619,맛집,경북 경주시 천군동 산 30-4,CE7,카페,음식점 > 카페 > 테마카페 > 디저트카페,NaN,1498815133,NaN,BBS,http://place.map.kakao.com/1498815133,경북 경주시 보불로 391,129.290743,35.825261
1620,맛집,경북 경주시 신평동 400-1,CE7,카페,음식점 > 카페 > 커피전문점 > 달콤커피,NaN,269508741,NaN,달콤커피 경주대명리조트점,http://place.map.kakao.com/269508741,경북 경주시 보문로 402-12,129.281856,35.846577
1621,맛집,경북 경주시 북군동 11-1,CE7,카페,음식점 > 카페 > 테마카페 > 키즈카페,NaN,82342856,054-748-8024,켄싱턴리조트 경주보문점 코코몽키즈플레이존,http://place.map.kakao.com/82342856,경북 경주시 보문로 182-29,129.268475,35.860711
